In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

print("✅ All libraries installed and working properly!")

✅ All libraries installed and working properly!


In [2]:
# ============================================
# 📘 Project 4: Prediction of Agriculture Crop Production in India
# Step 1 – Data Loading, Cleaning, and Merging
# ============================================

# ✅ Import libraries
import pandas as pd
import numpy as np

print("Libraries imported successfully!")

# ============================================
# STEP 1: LOAD ALL FILES
# ============================================

# 🔹 Change paths to your actual file names if needed
file1 = r"C:\Users\Admin\Desktop\Crop_Production_Prediction_Project\data\raw\Project4_Ag_Prediction of Agriculture Crop Production In India\datafile (1).csv"
file2 = r"C:\Users\Admin\Desktop\Crop_Production_Prediction_Project\data\raw\Project4_Ag_Prediction of Agriculture Crop Production In India\datafile (2).csv"
file3 = r"C:\Users\Admin\Desktop\Crop_Production_Prediction_Project\data\raw\Project4_Ag_Prediction of Agriculture Crop Production In India\datafile (3).csv"
file4 = r"C:\Users\Admin\Desktop\Crop_Production_Prediction_Project\data\raw\Project4_Ag_Prediction of Agriculture Crop Production In India\datafile.csv"
file5 = r"C:\Users\Admin\Desktop\Crop_Production_Prediction_Project\data\raw\Project4_Ag_Prediction of Agriculture Crop Production In India\produce.csv"

# Read files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)
df5 = pd.read_csv(file5)

print("✅ All 5 CSVs loaded successfully!")
print("Shapes:")
print("File1:", df1.shape)
print("File2:", df2.shape)
print("File3:", df3.shape)
print("File4:", df4.shape)
print("File5:", df5.shape)

# ============================================
# STEP 2: CLEAN AND RENAME COLUMNS
# ============================================

# --- File 1: Cost & Yield Data ---
df1.columns = ['Crop','State','Cost_A2_FL','Cost_C2','Cost_Prod_C2','Yield']
df1['Crop'] = df1['Crop'].str.strip().str.title()
df1['State'] = df1['State'].str.strip().str.title()

# --- File 2: Yearly Crop Production ---
df2 = df2.rename(columns=lambda x: x.strip())
df2['Crop'] = df2['Crop'].str.strip().str.title()

# Reshape year-wise columns into single Year column
df2_melted = df2.melt(id_vars=['Crop'], var_name='Year', value_name='Production')
df2_melted = df2_melted.dropna(subset=['Production'])
df2_melted['Year'] = df2_melted['Year'].str.extract('(\d{4}-\d{2})')

# --- File 3: Variety and Season ---
df3 = df3.rename(columns={
    'Crop':'Crop',
    'Variety':'Variety',
    'Season/ duration in days':'Season_Duration',
    'Recommended Zone':'Recommended_Zone'
})
df3 = df3.drop(columns=[col for col in df3.columns if 'Unnamed' in col])
df3['Crop'] = df3['Crop'].str.strip().str.title()

# --- File 4: Crop Index over Years ---
df4 = df4.rename(columns=lambda x: x.strip())
df4['Crop'] = df4['Crop'].str.strip().str.title()

# Convert to long format
df4_melted = df4.melt(id_vars=['Crop'], var_name='Year', value_name='Index')
df4_melted['Year'] = df4_melted['Year'].str.extract('(\d{4}-\d{2})')

# --- File 5: National Production Data ---
df5 = df5.rename(columns=lambda x: x.strip())
df5 = df5.drop(columns=['Frequency'], errors='ignore')
df5 = df5.rename(columns={'Particulars':'Category'})
df5['Category'] = df5['Category'].str.strip().str.title()

print("✅ Column cleaning and renaming complete!")

# ============================================
# STEP 3: MERGE RELATED FILES
# ============================================

# Merge df2 (Production) with df1 (Cost & Yield)
merged = pd.merge(df2_melted, df1, on='Crop', how='left')

# Merge df3 (Variety & Season)
merged = pd.merge(merged, df3, on='Crop', how='left')

# Merge df4 (Index/Trend data)
merged = pd.merge(merged, df4_melted, on=['Crop','Year'], how='left')

print("✅ All relevant datasets merged successfully!")
print("Merged shape:", merged.shape)

# ============================================
# STEP 4: CLEAN FINAL DATASET
# ============================================

# Drop duplicates and handle missing values
merged.drop_duplicates(inplace=True)
merged.fillna({'Cost_A2_FL':0,'Cost_C2':0,'Cost_Prod_C2':0,'Yield':0,'Index':0}, inplace=True)

# Convert Year to a numeric or simpler format (e.g., 2010)
merged['Year'] = merged['Year'].str[:4]
merged['Year'] = pd.to_numeric(merged['Year'], errors='coerce')

print("✅ Final cleaning done!")
print("Final dataset shape:", merged.shape)

# ============================================
# STEP 5: SAVE FINAL PROCESSED FILE
# ============================================

output_path = "final_crop_dataset.csv"
merged.to_csv(output_path, index=False)

print(f"🎯 Final merged dataset saved as: {output_path}")

# ============================================
# STEP 6: QUICK CHECK
# ============================================

print("\nPreview of final dataset:")
print(merged.head(10))


Libraries imported successfully!
✅ All 5 CSVs loaded successfully!
Shapes:
File1: (49, 6)
File2: (55, 16)
File3: (78, 5)
File4: (13, 9)
File5: (429, 25)
✅ Column cleaning and renaming complete!
✅ All relevant datasets merged successfully!
Merged shape: (2415, 12)
✅ Final cleaning done!
Final dataset shape: (2385, 12)
🎯 Final merged dataset saved as: final_crop_dataset.csv

Preview of final dataset:
               Crop  Year  Production           State  Cost_A2_FL   Cost_C2  \
0  Total Foodgrains  2006       158.8             NaN         0.0      0.00   
1              Rice  2006       200.8             NaN         0.0      0.00   
2             Wheat  2006       131.6  Madhya Pradesh     12464.4  22489.75   
3             Wheat  2006       131.6  Madhya Pradesh     12464.4  22489.75   
4             Wheat  2006       131.6  Madhya Pradesh     12464.4  22489.75   
5             Wheat  2006       131.6  Madhya Pradesh     12464.4  22489.75   
6             Wheat  2006       131.6  Madhya

In [3]:
import os
os.path.exists("C:/Users/Admin/Desktop/Crop_Production_Prediction_Project/data/processed/final_crop_dataset.csv")

True